In [1]:
#Importamos las librerias necesarias
import os  
import pandas as pd
import numpy as np
import json
import unicodedata
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
#Cargamos los datos desde los archivos
cards_data = pd.read_csv('cards_data.csv')
transactions_data = pd.read_csv('transactions_data.csv')
users_data = pd.read_csv('users_data.csv')
#mcc_codes = pd.read_csv('mcc_codes.json')
#train_fraud_labels =  pd.read_csv('train_fraud_labels.json')

In [3]:
#Pasamos los archivos json a un formato de dataframe

# 1. Abrir el archivo JSON
with open("mcc_codes.json", "r", encoding="utf-8") as f:
    mcc_dict = json.load(f)

# 2. Convertir el diccionario a una lista de tuplas y luego a DataFrame
rows = []
for code_str, desc in mcc_dict.items():
    # Si prefieres que el código mcc sea entero, puedes hacer: int(code_str)
    rows.append({
        "mcc": int(code_str),               # convierte "5812" → 5812
        "mcc_description": desc             # por ejemplo, "Eating Places and Restaurants"
    })

mcc_codes_df = pd.DataFrame(rows)

# 3. (Opcional) Ordenar por código
mcc_codes_df = mcc_codes_df.sort_values("mcc").reset_index(drop=True)

# 4. Ver resultado
print(mcc_codes_df.shape)
print(mcc_codes_df.head(10))

(109, 2)
    mcc                                  mcc_description
0  1711  Heating, Plumbing, Air Conditioning Contractors
1  3000                                       Steelworks
2  3001                     Steel Products Manufacturing
3  3005                  Miscellaneous Metal Fabrication
4  3006          Miscellaneous Fabricated Metal Products
5  3007                    Coated and Laminated Products
6  3008                          Steel Drums and Barrels
7  3009             Fabricated Structural Metal Products
8  3058             Tools, Parts, Supplies Manufacturing
9  3066                             Miscellaneous Metals


In [4]:
import json
import pandas as pd

# 1. Abrir el JSON
with open("train_fraud_labels.json", "r", encoding="utf-8") as f:
    labels_dict = json.load(f)


# 2. Extraer el diccionario interno que está bajo la clave "target"
inner = labels_dict["target"]

# 3. Convertir ese sub-diccionario a lista de filas
rows = []
for id_str, label_str in inner.items():
    rows.append({
        "id": int(id_str),               # convierte "10649266" → 10649266
        "is_fraud": 1 if label_str.lower() == "yes" else 0
    })

fraud_labels_df = pd.DataFrame(rows)

# 4. (Opcional) Ordenar por 'id'
fraud_labels_df = fraud_labels_df.sort_values("id").reset_index(drop=True)

# 5. Ver resultado
print(fraud_labels_df.shape)
print(fraud_labels_df.head(10))


(8914963, 2)
        id  is_fraud
0  7475327         0
1  7475328         0
2  7475329         0
3  7475332         0
4  7475333         0
5  7475335         0
6  7475338         0
7  7475339         0
8  7475340         0
9  7475341         0


In [5]:
#Inspección inicial de los datos cards_data
print("Dimensiones iniciales de 'transactions':", cards_data.shape)
print(cards_data.dtypes)
print(cards_data.head(10))

Dimensiones iniciales de 'transactions': (6146, 13)
id                        int64
client_id                 int64
card_brand               object
card_type                object
card_number               int64
expires                  object
cvv                       int64
has_chip                 object
num_cards_issued          int64
credit_limit             object
acct_open_date           object
year_pin_last_changed     int64
card_on_dark_web         object
dtype: object
     id  client_id  card_brand        card_type       card_number  expires  \
0  4524        825        Visa            Debit  4344676511950444  12/2022   
1  2731        825        Visa            Debit  4956965974959986  12/2020   
2  3701        825        Visa            Debit  4582313478255491  02/2024   
3    42        825        Visa           Credit  4879494103069057  08/2024   
4  4659        825  Mastercard  Debit (Prepaid)  5722874738736011  03/2009   
5  4537       1746        Visa           Credit  4

In [6]:
#Inspección inicial de los datos transactions_data
print("Dimensiones iniciales de 'transactions':", transactions_data.shape)
print(transactions_data.dtypes)
print(transactions_data.head(10))

Dimensiones iniciales de 'transactions': (13305915, 12)
id                  int64
date               object
client_id           int64
card_id             int64
amount             object
use_chip           object
merchant_id         int64
merchant_city      object
merchant_state     object
zip               float64
mcc                 int64
errors             object
dtype: object
        id                 date  client_id  card_id   amount  \
0  7475327  2010-01-01 00:01:00       1556     2972  $-77.00   
1  7475328  2010-01-01 00:02:00        561     4575   $14.57   
2  7475329  2010-01-01 00:02:00       1129      102   $80.00   
3  7475331  2010-01-01 00:05:00        430     2860  $200.00   
4  7475332  2010-01-01 00:06:00        848     3915   $46.41   
5  7475333  2010-01-01 00:07:00       1807      165    $4.81   
6  7475334  2010-01-01 00:09:00       1556     2972   $77.00   
7  7475335  2010-01-01 00:14:00       1684     2140   $26.46   
8  7475336  2010-01-01 00:21:00        335

In [7]:
#Inspección inicial de los datos cards_data
print("Dimensiones iniciales de 'transactions':", users_data.shape)
print(users_data.dtypes)
print(users_data.head(10))

Dimensiones iniciales de 'transactions': (2000, 14)
id                     int64
current_age            int64
retirement_age         int64
birth_year             int64
birth_month            int64
gender                object
address               object
latitude             float64
longitude            float64
per_capita_income     object
yearly_income         object
total_debt            object
credit_score           int64
num_credit_cards       int64
dtype: object
     id  current_age  retirement_age  birth_year  birth_month  gender  \
0   825           53              66        1966           11  Female   
1  1746           53              68        1966           12  Female   
2  1718           81              67        1938           11  Female   
3   708           63              63        1957            1  Female   
4  1164           43              70        1976            9    Male   
5    68           42              70        1977           10    Male   
6  1075          